##### Loading libraries and data importing

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
data=pd.read_csv("/home/pinky/chem/data.csv")
data

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
0,NCGC00178831-03,5.436720e+07,13.053,2.176,3.194,23.112,15.868,1.496,15.127,12.592,...,2687.469,9241.018,115.371,-915.496,-39.983,290.078,2301.941,59.492,88.147,3.708
1,NCGC00166114-03,1.268818e+07,22.123,2.065,3.137,21.033,13.718,1.937,13.187,11.951,...,2184.384,3234.199,194.740,-1029.609,-34.205,235.360,1244.323,82.906,134.852,4.131
2,NCGC00263563-01,3.076932e+06,13.085,2.154,3.207,46.896,29.958,3.806,30.105,25.569,...,13803.524,76582.899,238.004,-4358.946,-106.537,868.685,15909.444,135.335,216.852,5.075
3,NCGC00013058-02,7.168569e+07,12.832,2.029,3.380,51.086,32.045,1.806,29.090,21.603,...,13807.345,50498.175,226.312,-2785.555,-61.923,763.288,9394.859,125.509,238.265,4.640
4,NCGC00167516-01,7.989702e+06,12.936,2.124,3.573,70.295,46.402,3.604,42.132,32.570,...,43231.286,163659.229,850.869,-21136.699,-367.122,1798.703,44681.209,362.168,317.901,7.845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,1.428572e+07,14.255,2.000,2.628,9.259,6.309,0.157,5.468,4.484,...,306.364,1435.590,76.419,-190.192,-13.757,100.624,250.402,19.275,29.148,2.581
12056,NCGC00261245-01,1.193182e+07,13.674,2.061,2.920,21.142,15.382,1.201,12.713,10.576,...,2528.642,12293.627,94.878,-595.491,-22.275,324.131,2034.439,49.446,93.636,3.666
12057,NCGC00260828-01,1.081800e+01,12.374,2.045,3.128,33.242,20.457,0.806,19.711,14.799,...,9171.300,44070.070,267.400,-2656.568,-104.039,874.679,8689.849,144.294,91.670,8.054
12058,NCGC00260687-01,3.229000e+00,12.543,2.267,2.700,10.251,7.381,0.587,6.455,5.857,...,391.790,1815.417,39.578,-105.234,-9.967,146.565,389.732,23.879,28.201,2.954


##### Removing correlated variables

In order to improve speed and reducing storage uses correlated variables have been removed because correlated variables does not have extra information.

In [63]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value 
corr_matrix = data.corr().abs()
corr_matrix.head()

,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
AW,1.000000,0.158051,0.354390,0.035270,0.174191,0.149417,0.032700,0.158637,0.079242,0.215184,...,0.088828,0.079596,0.153544,0.097605,0.198085,0.092218,0.090137,0.155466,0.049872,0.069629
AWeight,0.158051,1.000000,0.412161,0.306281,0.148869,0.169010,0.121028,0.146384,0.158145,0.036075,...,0.057599,0.080419,0.013899,0.025670,0.035924,0.140745,0.063173,0.100396,0.019448,0.152597
Arto,0.354390,0.412161,1.000000,0.760954,0.405603,0.458151,0.474185,0.467011,0.550558,0.291049,...,0.166604,0.146509,0.291548,0.096393,0.153367,0.259401,0.118078,0.192370,0.039171,0.401940
BertzCT,0.035270,0.306281,0.760954,1.000000,0.737756,0.761156,0.605124,0.773977,0.792567,0.609553,...,0.397326,0.367001,0.619279,0.289741,0.471239,0.517173,0.340893,0.492850,0.007237,0.686158
Chi0,0.174191,0.148869,0.405603,0.737756,1.000000,0.993443,0.760671,0.988669,0.953568,0.813185,...,0.765717,0.751536,0.820781,0.575218,0.716676,0.828039,0.710015,0.777238,0.002843,0.785810


In [64]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

/tmp/ipykernel_7301/1262671735.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
AW,NaN,0.158051,0.354390,0.035270,0.174191,0.149417,0.032700,0.158637,0.079242,0.215184,...,0.088828,0.079596,0.153544,0.097605,0.198085,0.092218,0.090137,0.155466,0.049872,0.069629
AWeight,NaN,NaN,0.412161,0.306281,0.148869,0.169010,0.121028,0.146384,0.158145,0.036075,...,0.057599,0.080419,0.013899,0.025670,0.035924,0.140745,0.063173,0.100396,0.019448,0.152597
Arto,NaN,NaN,NaN,0.760954,0.405603,0.458151,0.474185,0.467011,0.550558,0.291049,...,0.166604,0.146509,0.291548,0.096393,0.153367,0.259401,0.118078,0.192370,0.039171,0.401940
BertzCT,NaN,NaN,NaN,NaN,0.737756,0.761156,0.605124,0.773977,0.792567,0.609553,...,0.397326,0.367001,0.619279,0.289741,0.471239,0.517173,0.340893,0.492850,0.007237,0.686158
Chi0,NaN,NaN,NaN,NaN,NaN,0.993443,0.760671,0.988669,0.953568,0.813185,...,0.765717,0.751536,0.820781,0.575218,0.716676,0.828039,0.710015,0.777238,0.002843,0.785810


In [65]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

There are 442 columns to remove.


In [66]:
data = data.drop(columns = to_drop)

In [67]:
data

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi10,Chi3c,Chi3ch,Chi4c,...,RDFP5,RDFP6,RDFP7,RDFP8,RDFP9,RNCS,RPCS,TASA,WNSA1,grav
0,NCGC00178831-03,5.436720e+07,13.053,2.176,3.194,23.112,1.496,2.619,0.0,0.000,...,4.809,18.623,23.764,27.692,31.024,0,83.808,290.190,115.371,88.147
1,NCGC00166114-03,1.268818e+07,22.123,2.065,3.137,21.033,1.937,2.502,0.0,0.000,...,33.221,38.383,27.289,25.492,46.996,0,46.958,421.783,194.740,134.852
2,NCGC00263563-01,3.076932e+06,13.085,2.154,3.207,46.896,3.806,7.819,0.0,0.762,...,37.670,45.713,52.060,66.850,87.861,0,0.000,804.917,238.004,216.852
3,NCGC00013058-02,7.168569e+07,12.832,2.029,3.380,51.086,1.806,5.222,0.0,0.000,...,33.235,53.165,58.062,65.033,76.869,0,700.786,823.226,226.312,238.265
4,NCGC00167516-01,7.989702e+06,12.936,2.124,3.573,70.295,3.604,7.002,0.0,0.000,...,34.154,57.701,51.678,93.055,93.111,0,491.912,938.923,850.869,317.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,1.428572e+07,14.255,2.000,2.628,9.259,0.157,0.758,0.0,0.000,...,5.429,8.886,3.821,7.416,2.770,0,254.365,280.462,76.419,29.148
12056,NCGC00261245-01,1.193182e+07,13.674,2.061,2.920,21.142,1.201,1.082,0.0,0.000,...,18.126,36.615,25.860,33.063,31.819,0,268.098,550.993,94.878,93.636
12057,NCGC00260828-01,1.081800e+01,12.374,2.045,3.128,33.242,0.806,4.733,0.0,0.408,...,24.295,27.286,17.842,27.651,28.422,0,605.744,812.081,267.400,91.670
12058,NCGC00260687-01,3.229000e+00,12.543,2.267,2.700,10.251,0.587,0.810,0.0,0.000,...,3.226,3.995,3.355,3.289,6.596,0,80.674,309.909,39.578,28.201


In [68]:
data_missing = (data.isnull().sum() / len(data)).sort_values(ascending = False)
data_missing.head()

Unnamed: 0    0.0
MoRSEM19      0.0
MoRSEM17      0.0
MoRSEM16      0.0
MoRSEM15      0.0
dtype: float64

##### Identify missing values

In [69]:
# Identify missing values above threshold
data_missing = data_missing.index[data_missing > 0.50]

In [70]:
print('There are %d columns with more than 50%% missing values' % len(data_missing))

There are 0 columns with more than 50% missing values


In [71]:
data.replace(np.nan, 0)

,Unnamed: 0,AW,AWeight,Arto,BertzCT,Chi0,Chi10,Chi3c,Chi3ch,Chi4c,...,RDFP5,RDFP6,RDFP7,RDFP8,RDFP9,RNCS,RPCS,TASA,WNSA1,grav
0,NCGC00178831-03,5.436720e+07,13.053,2.176,3.194,23.112,1.496,2.619,0.0,0.000,...,4.809,18.623,23.764,27.692,31.024,0,83.808,290.190,115.371,88.147
1,NCGC00166114-03,1.268818e+07,22.123,2.065,3.137,21.033,1.937,2.502,0.0,0.000,...,33.221,38.383,27.289,25.492,46.996,0,46.958,421.783,194.740,134.852
2,NCGC00263563-01,3.076932e+06,13.085,2.154,3.207,46.896,3.806,7.819,0.0,0.762,...,37.670,45.713,52.060,66.850,87.861,0,0.000,804.917,238.004,216.852
3,NCGC00013058-02,7.168569e+07,12.832,2.029,3.380,51.086,1.806,5.222,0.0,0.000,...,33.235,53.165,58.062,65.033,76.869,0,700.786,823.226,226.312,238.265
4,NCGC00167516-01,7.989702e+06,12.936,2.124,3.573,70.295,3.604,7.002,0.0,0.000,...,34.154,57.701,51.678,93.055,93.111,0,491.912,938.923,850.869,317.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,1.428572e+07,14.255,2.000,2.628,9.259,0.157,0.758,0.0,0.000,...,5.429,8.886,3.821,7.416,2.770,0,254.365,280.462,76.419,29.148
12056,NCGC00261245-01,1.193182e+07,13.674,2.061,2.920,21.142,1.201,1.082,0.0,0.000,...,18.126,36.615,25.860,33.063,31.819,0,268.098,550.993,94.878,93.636
12057,NCGC00260828-01,1.081800e+01,12.374,2.045,3.128,33.242,0.806,4.733,0.0,0.408,...,24.295,27.286,17.842,27.651,28.422,0,605.744,812.081,267.400,91.670
12058,NCGC00260687-01,3.229000e+00,12.543,2.267,2.700,10.251,0.587,0.810,0.0,0.000,...,3.226,3.995,3.355,3.289,6.596,0,80.674,309.909,39.578,28.201


In [72]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [73]:
data

,AW,AWeight,Arto,BertzCT,Chi0,Chi10,Chi3c,Chi3ch,Chi4c,Chi4ch,...,RDFP5,RDFP6,RDFP7,RDFP8,RDFP9,RNCS,RPCS,TASA,WNSA1,grav
0,5.436720e+07,13.053,2.176,3.194,23.112,1.496,2.619,0.0,0.000,0.0,...,4.809,18.623,23.764,27.692,31.024,0,83.808,290.190,115.371,88.147
1,1.268818e+07,22.123,2.065,3.137,21.033,1.937,2.502,0.0,0.000,0.0,...,33.221,38.383,27.289,25.492,46.996,0,46.958,421.783,194.740,134.852
2,3.076932e+06,13.085,2.154,3.207,46.896,3.806,7.819,0.0,0.762,0.0,...,37.670,45.713,52.060,66.850,87.861,0,0.000,804.917,238.004,216.852
3,7.168569e+07,12.832,2.029,3.380,51.086,1.806,5.222,0.0,0.000,0.0,...,33.235,53.165,58.062,65.033,76.869,0,700.786,823.226,226.312,238.265
4,7.989702e+06,12.936,2.124,3.573,70.295,3.604,7.002,0.0,0.000,0.0,...,34.154,57.701,51.678,93.055,93.111,0,491.912,938.923,850.869,317.901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,1.428572e+07,14.255,2.000,2.628,9.259,0.157,0.758,0.0,0.000,0.0,...,5.429,8.886,3.821,7.416,2.770,0,254.365,280.462,76.419,29.148
12056,1.193182e+07,13.674,2.061,2.920,21.142,1.201,1.082,0.0,0.000,0.0,...,18.126,36.615,25.860,33.063,31.819,0,268.098,550.993,94.878,93.636
12057,1.081800e+01,12.374,2.045,3.128,33.242,0.806,4.733,0.0,0.408,0.0,...,24.295,27.286,17.842,27.651,28.422,0,605.744,812.081,267.400,91.670
12058,3.229000e+00,12.543,2.267,2.700,10.251,0.587,0.810,0.0,0.000,0.0,...,3.226,3.995,3.355,3.289,6.596,0,80.674,309.909,39.578,28.201


##### Importing data labels

In [74]:
toxicity_label=pd.read_csv("/home/pinky/toxicity_labels.csv")
toxicity_label

,Unnamed: 0,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,NCGC00178831-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,NCGC00166114-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,NCGC00263563-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,NCGC00013058-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NCGC00167516-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12056,NCGC00261245-01,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
12057,NCGC00260828-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12058,NCGC00260687-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
toxicity_label_n=toxicity_label.fillna(0)

In [76]:
toxicity_label_n

,Unnamed: 0,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,NCGC00178831-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NCGC00166114-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NCGC00263563-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NCGC00013058-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NCGC00167516-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12055,NCGC00261292-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12056,NCGC00261245-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12057,NCGC00260828-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12058,NCGC00260687-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
toxicity_label_n.drop(['Unnamed: 0'], axis=1, inplace=True)

In [78]:
toxicity_label_n

,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
X = data.iloc[:,:-1] 
y = toxicity_label_n.iloc[:,-1]

In [80]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((9045, 358), (9045,), (3015, 358), (3015,))

 ##### LASSO regularization

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

s_f_m = SelectFromModel(LogisticRegression(C = 5, penalty = 'l2',max_iter=1000))
s_f_m.fit(X_train, y_train)
important_features = X_train.columns[(s_f_m.get_support())] 
print(important_features)

Index(['AW', 'AWeight', 'Arto', 'BertzCT', 'Chi0', 'Chi10', 'Chi3c', 'Chi3ch',
       'Chi4c', 'Chi4ch',
       ...
       'RDFP3', 'RDFP5', 'RDFP6', 'RDFP7', 'RDFP8', 'RDFP9', 'RNCS', 'RPCS',
       'TASA', 'WNSA1'],
      dtype='object', length=358)


/home/pinky/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### PCA analysis

##### 1. Standarization using z-score normalization 

In [82]:
from sklearn.preprocessing import StandardScaler

# Normalize the  data (center around 0 and scale to remove the variance).
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

##### 2. Constraction of Covariance matrix C

In [83]:
n = X.shape[0]
C = 1/(n-1) * np.dot(Xs.T,Xs)

##### 3. Computing the eigenvalues and eigenvectors 

In [84]:
# Eigen decomposition,

import numpy.linalg as linalg
d, V = linalg.eig(C)   #d is list of eigenvalues and V a matrix of eigenvectors

##### 4. Sorting the eigenvalues and eigenvectors in decreasing order

In [85]:
ind = np.argsort(d)[::-1]
d = d[ind]
V = V[:,ind]

##### Data restriction into two-dimension

In [86]:
Xr = np.dot(Xs,V[:,0:2])

##### Model evaluation- Optimized SVM-RBF

In [89]:
from sklearn.model_selection import StratifiedKFold  
from sklearn.model_selection import GridSearchCV 
from sklearn.gaussian_process.kernels import RBF
from sklearn import svm
from sklearn.svm import SVC 

In [91]:
for i in range(2):

 cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=i)



svc_grid = {'C': np.logspace(-2, 2, 5),
            'kernel': ['rbf'],
            'degree': [2,3,4],
            "gamma": np.logspace(-2,2,5)
           }

svc_gscv = GridSearchCV(svm.SVC(random_state = 35),
                        param_grid=svc_grid,
                        cv=3,
                        verbose=True)

svc_gscv.fit(X_train, y_train)
svc_tuned_score_rbf = svc_gscv.score(X_test, y_test)
svc_tuned_score_rbf_r = round((svc_tuned_score_rbf)*100,2)
svc_tuned_score_rbf_r

Fitting 3 folds for each of 75 candidates, totalling 225 fits


95.02